# Collecting Article Data from New York Time's Archive

Over 150 years of NYT articles can be collected from the Archive API. This notebook extracts some useful information for some analysis for a period of time.

#### Importing necessary libraries

In [1]:
import pandas as pd
import json
import configparser
import requests
import time

#### Reading the API key from Configparser

In [2]:
configs = configparser.ConfigParser()
configs.read('News_API.ini')
API_KEY = configs['news_API']['nytimes_archive']

#### Simple Data Extractor

The NYT allows to extract one month of archive data in one call and the data is in JSON file format. A simple function is created to extract one JSON file contains one month of articles.

In [3]:
def send_request(yr, month, api_key):
  '''Sends a request to the NYT Archive API for a given year and month, and receive the response'''
    
  base_url = 'https://api.nytimes.com/svc/archive/v1'
  full_url = base_url + '/' + yr + '/' + month + '.json?api-key=' + api_key
    
  try:
    response = requests.get(full_url).json()
  except Exception:
    return None
  return response

Sample output of `send_request()` function.

In [4]:
nyt_articles = send_request(str(2021), str(6), API_KEY)
len(nyt_articles['response']['docs'])

4260

Information about 4260 articles comprised in the June 2021 archive. First dictionary item is printed to see what data is available for further extraction and analysis.

In [5]:
nyt_articles['response']['docs'][0]

{'abstract': 'In Israel’s newspapers — as fractured as its electorate — the opinions on the possible fall of Prime Minister Benjamin Netanyahu were as emphatic as they were varied.',
 'web_url': 'https://www.nytimes.com/2021/05/31/world/middleeast/israeli-media-netanyahu-bennett.html',
 'snippet': 'In Israel’s newspapers — as fractured as its electorate — the opinions on the possible fall of Prime Minister Benjamin Netanyahu were as emphatic as they were varied.',
 'lead_paragraph': 'After more than 12 consecutive years at the helm of Israel’s government, Prime Minister Benjamin Netanyahu’s hold on power may be coming to a close.',
 'source': 'The New York Times',
 'multimedia': [{'rank': 0,
   'subtype': 'xlarge',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2021/05/31/world/31israel-reaction/merlin_188519604_9d0ecaa9-cca2-4523-ae7c-63df787502c5-articleLarge.jpg',
   'height': 400,
   'width': 600,
   'subType': 'xlarge',
   'crop_name': 'articleLarge',

### Extracting Useful Information From the Data Dictionary

Extracting following information:
pub_date, document_type, news_desk, section_name, abstract, word_count, web_url

In [7]:
data = {'pub_date': [],
        'document_type':[],
        'news_desk':[],
        'section_name': [],
        'abstract': [],
        'word_count':[],
        'web_url': []}

articles = nyt_articles['response']['docs']

for article in articles:
  data['abstract'].append(article['abstract'])
  data['document_type'].append(article['document_type'])
  data['news_desk'].append(article['news_desk'])
  data['section_name'].append(article['section_name'])
  data['pub_date'].append(pd.to_datetime(article['pub_date'], format='%Y%m%d %H:%M:%S.%f').date())
  data['web_url'].append(articles['web_url'])
  data['word_count'].append(article['word_count'])

TypeError: list indices must be integers or slices, not str

In [62]:
# incase if column length error pops up
# df = pd.DataFrame.from_dict(data, orient='index')
# df = df.transpose()

In [63]:
df = pd.DataFrame(data)
df.head()

,pub_date,document_type,news_desk,section_name,abstract,word_count,web_url
0,2021-06-01,article,Foreign,World,In Israel’s newspapers — as fractured as its e...,984,https://www.nytimes.com/interactive/2021/05/31...
1,2021-06-01,article,Sports,Sports,"Playing in a night session in Paris, Williams ...",722,https://www.nytimes.com/interactive/2021/05/31...
2,2021-06-01,article,Express,U.S.,A bill that passed the General Assembly with b...,880,https://www.nytimes.com/interactive/2021/05/31...
3,2021-06-01,article,Games,Crosswords & Games,Finn Vigeland offers us a refreshing snack of ...,658,https://www.nytimes.com/interactive/2021/05/31...
4,2021-06-01,multimedia,Climate,Climate,There are few things Americans can agree on th...,0,https://www.nytimes.com/interactive/2021/05/31...


Need to add headline, keywords, person. Need to check subsection_name